In [1]:
import glob
import os
import csv
from scipy.interpolate import interp1d
import numpy as np
import pylab
from scipy.integrate import fixed_quad
from __future__ import division
import pickle

#MATERIALS
#Extract the files .CSV in the home archive that supposely are the spectrum information 
#of the materials

# create the list of files
materials = glob.glob('materials/*'); 
num_mat=len(materials);

mat_int=[];           #store the intensities of the materials csv files
mat_wl=[];            #store the wavelenght of the materials csv files
name_material=[];     #names of the files where the material's spectrum are

i_mat=0;
for x in materials:
    material=open(x,'rU');
    name_mat=os.path.basename(x).replace('.CSV','').replace('.csv','');
    name_material.append(name_mat);
    csv_material=csv.reader(material);
    mat_int.append([]);
    mat_wl.append([]);
    for row in csv_material:
        mat_wl[i_mat].append(float(row[0]));
        mat_int[i_mat].append(float(row[1]));
    i_mat+=1;
    material.close();
    
#make continuous the intensity_mat function
f_mat=range(num_mat);
f_mat[:]=[[] for x in f_mat];
for i in range(num_mat):
    f_mat[i]=interp1d(mat_wl[i],mat_int[i]);

#Sensor's radiometry
    
#open the csv file in pyhton
responses = glob.glob('responses/*');
num_responses=len(responses);
name_responses=[];

for x in responses:
    resp=open(x,'rU');
    csv_response=csv.reader(resp);
    name_resp=os.path.basename(x).replace('.csv','').replace('.CSV','');
    name_responses.append(name_resp);
    name_bands=[];
    #Value used to read the titles or first lines of the satellites' spectrums
    safe=0;
    #put the sensor's information in lists
    wavelenght=[];
    for row in csv_response:
        if safe==0:
            #Here assign the name of the bands as well as the number of bands
            num_bands=len(row)-1;
            for i in range(num_bands):
                name_bands.append(row[i+1]);
            band=range(num_bands);
            band[:]=[[] for x in band];
            safe=1;
        else:
            wavelenght.append(float(row[0]));
            for i in range(num_bands):
                band[i].append(float(row[i+1]));  

    #should i to close the file
    resp.close();
 
    #Make continuous the sensors' spectrum
    f_sen=range(num_bands);
    for i in range(num_bands):
        f_sen[i]=interp1d(wavelenght,band[i]);

        
    # COEFFICIENTS

    coef=np.zeros((num_mat,num_bands));
    result=range(num_bands);
    #integrate to obtain the coefficients for every material in every band
    for i in range(num_mat):   #count over the materials' spectrum
        for j in range(num_bands):    #count over the satellites bands
            xmin=max(min(wavelenght),min(mat_wl[i]));
            xmax=min(max(wavelenght),max(mat_wl[i]));
        
            #Find the range of integration and write it down in the answer
        
            Result=fixed_quad(lambda x:f_sen[j](x)*f_mat[i](x),xmin,xmax,n=400);
            coef[i,j]=float(Result[0]); 
        coef_max=max(coef[i,:]);
        coef[i,:]/=coef_max;
        # To store the values of the coefficients
        coef_temp=open('coefficients/coef.'+name_resp,'w');
        pickle.dump(coef,coef_temp);
        coef_temp.close();
        # To store the names of materials whose coefficients were stored 
        name_temp=open('coefficients/mat.'+name_resp,'w');
        pickle.dump(name_material,name_temp);
        name_temp.close();
    

In [3]:
print coef

[[  1.72481212e-01   1.32125520e-06   2.41578861e-06   2.35523310e-07
    1.82171954e-05   1.61734039e-02   3.95032888e-02   7.53849113e-02
    6.31420770e-02   2.87652810e-01   1.66054915e-01   4.20990832e-01
    1.00000000e+00   5.28378062e-03   1.05768375e-02   8.42005172e-02
    2.64149839e-02]
 [  1.00000000e+00   2.22669856e-06   2.15256162e-06   1.50765021e-06
    4.70661317e-05   4.00971051e-01   2.91542995e-01   1.11681560e-01
    4.18874414e-02   1.14643158e-01   5.54293576e-02   8.36862139e-02
    3.69933456e-01   1.34056590e-03   1.88784132e-03   2.74083572e-03
    1.72682309e-03]
 [  1.00000000e+00   3.41704637e-06   4.89891654e-06   1.50344016e-05
    1.45224686e-01   3.91376336e-01   2.06279008e-01   1.33501656e-01
    1.37572147e-01   9.36452128e-02   1.68828730e-02   2.35121508e-02
    9.79888574e-02   2.36920849e-03   2.78446197e-03   2.81327832e-03
    3.06095391e-03]
 [  1.00000000e+00   4.68390112e-06   1.00728578e-05   3.39186783e-05
    3.98424117e-01   3.5693109